In [0]:
%matplotlib inline

import os
import multiprocessing
from multiprocessing import Pool

import numpy as np
import pandas as pd
from tqdm import tqdm
import mlcrate as mlc
from bayes_opt import BayesianOptimization

from trackml.dataset import load_event
from trackml.dataset import load_dataset
from trackml.score import score_event

from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.cluster.dbscan_ import dbscan
from sklearn.neighbors import KDTree

import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import ipyvolume as ipv
import ipywidgets as widgets

class Config():
    ROOT_PATH = '[PATH]'

    DATA_FOLDERS = {
        'train': 'train_100_events', 
        'test': 'test'
    }
    
    EVENT = 'event000001000'

    OUTPUT_FOLDER = './data'
    
    SPLIT = 0.75
    THETA = 75
    
    CORES = 96

config = Config()

EPS = 1e-10

event_path = os.path.join(config.ROOT_PATH, config.DATA_FOLDERS['train'], config.EVENT)
hits, cells, particles, truth = load_event(event_path)

def match_hits_and_particles(hits, particles, truth):
    particles = particles[particles.particle_id != 0]

    particle_ids = particles.particle_id.tolist()

    truth_of_particles = truth[truth.particle_id.isin(particle_ids)]

    hits_and_particles = pd.merge(truth_of_particles, hits, on='hit_id', how='inner')

    return hits_and_particles

merged = match_hits_and_particles(hits, particles, truth)

merged = merged.sort_values(by=['z'])
hits = hits.sort_values(by=['z'])

/home/bilal/track-ml/track-ml/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [0]:
class Model(object):
    def __init__(self, hits, truth, event, epsilon, weights, niter, stepiter, nvz, stepvz, scalevz, visualize=False, zshift=False, extend=False, test=False, rounds=0):
        self.event = event
        
        self.hits = hits.copy()
        self.truth = truth.copy()
        
        self.epsilon = epsilon
        
        self.weights = weights
        
        self.niter = int(niter)
        self.stepiter = stepiter
        
        self.nvz = int(nvz)
        self.stepvz = int(stepvz)
        self.scalevz = scalevz
        
        self.visualize = visualize
        self.extend = extend
        self.zshift = zshift
        self.test = test
        
        self.rounds = rounds
        
    def _preprocess(self):
        self.hits['r'] = np.sqrt(np.square(self.hits['x']) + np.square(self.hits['y']) + np.square(self.hits['z']))
        self.hits['rt'] = np.sqrt(np.square(self.hits['x']) + np.square(self.hits['y']))

        self.hits['x2'] = self.hits['x'] / self.hits['rt']
        self.hits['y2'] = self.hits['y'] / self.hits['rt']
        
        self.hits['zt'] = self.hits['z'] / self.hits['rt']
        self.hits['zr'] = self.hits['z'] / self.hits['r']
        
        self.hits['xr'] = self.hits['x'] / self.hits['r']
        self.hits['yr'] = self.hits['y'] / self.hits['r']

        self.hits['rho'] = np.sqrt(np.square(self.hits['x']) + np.square(self.hits['y']))
        self.hits['phi'] = np.arctan2(self.hits['y'], self.hits['x'])
        self.hits['s'] = np.sin(self.hits['phi'])
        self.hits['c'] = np.cos(self.hits['phi'])
        
        self.hits['arctan2-yx'] = np.arctan2(self.hits['y'], self.hits['x'])
        
        self.hits['arctan2'] = np.arctan2(self.hits['z'], self.hits['rho'])

        self.hits['zt-arctan'] = np.arctan(self.hits['zt'])
        self.hits['zr-arctan'] = np.arctan(self.hits['zr'])

        self.hits['log1p'] = np.log1p(np.abs(self.hits['zr'])) * np.sign(self.hits['zr'])

        self.hits['r-abs'] = (self.hits['r'] - np.abs(self.hits['z'])) / self.hits['r']
        
    def _get_params(self):
        
        if self.zshift:
            params = []

            i = 0
            for vz in np.arange(-self.nvz, self.nvz+EPS, self.stepvz):
                count_ii = 0
                mm = 1

                for ii in np.arange(0, self.niter*self.stepiter, self.stepiter): 
                    count_ii += 1

                    for jj in range(2):
                        mm = mm*(-1)
                        eps_new = self.epsilon + count_ii * 10 ** (-5)

                        params.append((i, ii, mm, eps_new, vz))

                        i += 1
            self.params = params
        else:
            params = []

            i = 0
            count_ii = 0
            mm = 1

            for ii in np.arange(0, self.niter*self.stepiter, self.stepiter): 
                count_ii += 1

                for jj in range(2):
                    mm = mm*(-1)
                    eps_new = self.epsilon + count_ii * 10 ** (-5)

                    params.append((i, ii, mm, eps_new, 0))

                    i += 1
            self.params = params

    def _get_preds(self, param):
        i, ii, mm, eps, vz = param

        self.hits['a'] = self.hits['arctan2-yx'].values - np.nan_to_num(np.arccos(mm * ii * self.hits['rt'].values))
        self.hits['sina'] = np.sin(self.hits['a'].values)
        self.hits['cosa'] = np.cos(self.hits['a'].values)
        
        self.hits['zt'] = (self.hits['z'] + (vz * self.scalevz)) / self.hits['rt']
        self.hits['zr'] = (self.hits['z'] + (vz * self.scalevz)) / self.hits['r']
                    
        x = StandardScaler().fit_transform(np.column_stack([self.hits['sina'], self.hits['cosa'], self.hits['zt'], self.hits['zr'], self.hits['xr'], self.hits['yr']]))
        x = np.multiply(x, self.weights)

        _, preds = dbscan(x, eps=eps, min_samples=1, metric='euclidean', n_jobs=config.CORES)

        mlc.save(preds, f'iterations/iteration-{i}.pkl')
              
    def _train(self):
        self._get_params()
                
        print(f'Iterations: {len(self.params)}') 
        
        if self.test:
            pool = Pool(processes=config.CORES)
        else:
            pool = mlc.SuperPool()
        
        pool.map(self._get_preds, self.params)

        if self.test:
            pool.close()
        
    def _load_preds(self):

        if self.zshift:
            preds = np.zeros((int(((2 * self.nvz) / self.stepvz) * 2 * self.niter * 2), len(self.hits)))
            
            i = 0
            for vz in np.arange(-self.nvz, self.nvz+EPS, self.stepvz):
                for ii in np.arange(0, self.niter*self.stepiter, self.stepiter):
                    for _ in range(2):
                        preds[i] = mlc.load(f'iterations/iteration-{i}.pkl')
                        i += 1

            self.preds = preds
        else:
            preds = np.zeros((self.niter * 2, len(self.hits)))

            i = 0

            for ii in np.arange(0, self.niter*self.stepiter, self.stepiter):
                for _ in range(2):
                    preds[i] = mlc.load(f'iterations/iteration-{i}.pkl')
                    i += 1

            self.preds = preds
            
    def _add_count(self, i, l):        
        # unique: sorted unique values; reverse: indicies of unique to reconstruct l; count: num times each unique appears
        unique, reverse, count = np.unique(l, return_counts=True, return_inverse=True)

        # get num times each unique l appears
        c = count[reverse]

        # unassign any tracks with either 0 or > 20 hits    
        # TODO: reassign to tracks with an even number of hits

        c[np.where(l == 0)] = 0        
        c[np.where(c > 25)] = 0
        
        return (l, c)
    
    def _extend(self, submission, hits):
        for _ in range(self.rounds):
            df = submission.merge(hits,  on=['hit_id'], how='left')
            df = df.assign(d = np.sqrt( df.x**2 + df.y**2 + df.z**2 ))
            df = df.assign(r = np.sqrt( df.x**2 + df.y**2))
            df = df.assign(arctan2 = np.arctan2(df.z, df.r))

            for angle in range(-90,90,1):
                # OPTIMIZE: size of slice 
                df1 = df.loc[(df.arctan2>(angle-1.5)/180*np.pi) & (df.arctan2<(angle+1.5)/180*np.pi)]

                min_num_neighbours = len(df1)
                # OPTIMIZE: min num neighbors
                if min_num_neighbours<3: continue

                hit_ids = df1.hit_id.values
                x,y,z = df1[['x', 'y', 'z']].values.T
                r  = (x**2 + y**2)**0.5
                r  = r/1000
                a  = np.arctan2(y,x)
                c = np.cos(a)
                s = np.sin(a)
                
                # OPTIMIZE: Features
                tree = KDTree(np.column_stack([a,r]), metric='euclidean')

                track_ids = list(df1.track_id.unique())
                num_track_ids = len(track_ids)
                
                # OPTIMIZE: min length
                min_length=3

                for i in range(num_track_ids):
                    p = track_ids[i]
                    if p==0: continue

                    # investigate more

                    # get indexes of all hits with the same track_id
                    idx = np.where(df1.track_id==p)[0]

                    if len(idx)<min_length: continue

                    # sort indexes by increasing or decreasing z values
                    if angle>0:
                        idx = idx[np.argsort( z[idx])]
                    else:
                        idx = idx[np.argsort(-z[idx])]

                    ## start and end points  ##
                    idx0,idx1 = idx[0],idx[-1]
                    a0 = a[idx0]
                    a1 = a[idx1]
                    r0 = r[idx0]
                    rt = r[idx1]
                    c0 = c[idx0]
                    c1 = c[idx1]
                    s0 = s[idx0]
                    s1 = s[idx1]

                    da0 = a[idx[1]] - a[idx[0]] #direction 
                    dr0 = r[idx[1]] - r[idx[0]] 
                    divisor0 = (da0**2+dr0**2)**0.5 + 1e-6
                    direction0 = np.array([da0/divisor0,dr0/divisor0])

                    da1 = a[idx[-1]] - a[idx[-2]]
                    drt = r[idx[-1]] - r[idx[-2]]
                    divisort = (da1**2+drt**2)**0.5 + 1e-6
                    direction1 = np.array([da1/divisort,drt/divisort]) 

                    ## extend start point
                    
                    # Optimize: k
                    ns = tree.query([[a0,r0]], k=min(20,min_num_neighbours), return_distance=False)
                    ns = np.concatenate(ns)
                    da0ns = a0-a[ns]
                    dr0ns = r0-r[ns]
                    divisor0ns = (da0ns**2+dr0ns**2)**0.5
                    divisor0ns[np.where(divisor0ns==0)]=1

                    direction = np.array([da0ns/divisor0ns,dr0ns/divisor0ns]) 
                    ns = ns[(r0-r[ns]>0.01) &(np.matmul(direction.T,direction0)>0.9991)]

                    for n in ns:
                        df.loc[ df.hit_id==hit_ids[n],'track_id' ] = p 

                    ## extend end point
                    
                    # Optimize: k
                    ns = tree.query([[a1,rt]], k=min(20,min_num_neighbours), return_distance=False)
                    ns = np.concatenate(ns)
                    da1ns = a[ns]-a1
                    drtns = r[ns]-rt
                    divisortns = (da1ns**2+drtns**2)**0.5
                    divisortns[np.where(divisortns==0)]=1

                    direction = np.array([da1ns/divisortns,drtns/divisortns]) 
                    
                    # Optimize: values
                    ns = ns[(r[ns]-rt>0.01) &(np.matmul(direction.T,direction1)>0.9991)] 

                    for n in ns:
                        df.loc[ df.hit_id==hit_ids[n],'track_id' ] = p

            submission = df[['event_id', 'hit_id', 'track_id']]
        self.extended_submission = submission

    def _postprocess(self):
        self._load_preds()
        
        results = [self._add_count(i, l) for i, l in enumerate(self.preds)]

        preds, counts = results[0]

        for i in range(1, len(results)):
            l, c = results[i]

            idx = np.where((c - counts > 0))[0]
            preds[idx] = l[idx] + preds.max()
            counts[idx] = c[idx]

        self.hits['track_id'] = preds
        self.hits['event_id'] = self.event
        
        self.submission = self.hits
        
        if self.extend:
            extension_sub = self.hits[['hit_id', 'track_id', 'event_id']]
            extension_hits = self.hits[['hit_id', 'x', 'y', 'z', 'volume_id', 'layer_id', 'module_id']]
            self._extend(extension_sub, extension_hits)
            
            self.submission = self.extended_submission
            
    def predict(self):
        self._preprocess()
        self._train()
        self._postprocess()

        if self.visualize:
            fig = plt.figure()
            ax = fig.add_subplot(111, projection='3d')

            for particle in self.hits['track_id'][:100].unique():
                hit = self.hits[self.hits['track_id'] == particle]

                ax.scatter(hit.x, hit.y, hit.z, marker='o')
                ax.plot(hit.x, hit.y, hit.z)

            ax.set_xlabel('X Label')
            ax.set_ylabel('Y Label')
            ax.set_zlabel('Z Label')

            plt.show()
            
        if self.test:
            return self.submission
        else:
            score = score_event(self.truth, self.submission)

        return self.hits, self.submission, score

In [0]:
model = Model(hits, truth, '000001000', 0.004, [1.7, 1.7, 0.8, 0.2, 0.015, 0.015], 300, 0.000005, 50, 10, 0.1, visualize=False, zshift=True, extend=True, test=True, rounds=6)
sub = model.predict()
score = score_event(truth, sub)

In [0]:
submissions = []

for event_id, hits, cells in tqdm(load_dataset(os.path.join(config.ROOT_PATH, config.DATA_FOLDERS['test']), parts=['hits', 'cells'])):
    print(event_id)
    
    model = Model(hits, hits, event_id, 0.004, [1.5, 1.5, 0.73, 0.17, 0.027, 0.027], 300, 0.000005, 50, 10, 0.1, visualize=False, zshift=True, extend=True, test=True, rounds=6)
    sub = model.predict()

    submissions.append(sub)

    submission = pd.concat(submissions, axis=0)

    mlc.kaggle.save_sub(submission, 'submission.csv.gz')